# Preprocessing and Modelling Pre-Omicron

In [1]:

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import zscore, rankdata, kendalltau
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

import csv 
from collections import Counter
import datetime
import holidays
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy.sparse import csr_matrix

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

from lineartree import LinearTreeRegressor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, cross_validate
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor



In [4]:
# When I did my first modelling, I did not ensure I had the proper data type for all columns; this
# resulted in "Test Date" (previously encoded as an object) being assigned to cat_cols and subsquently onehot encoded,
# inflating my scores. This is resolved below. Scores were previously about 10 percentage points higher on simple
# linear models.
merged_df_pre_omi = pd.read_csv('data/pre_omi.csv')

merged_df_pre_omi['Test Date'] = pd.to_datetime(merged_df_pre_omi['Test Date'])

merged_df_pre_omi['School Status'] = merged_df_pre_omi['School Status'].astype('object')
merged_df_pre_omi['School-Aged Population']

0        72091.0
1       122994.0
2       117865.0
3       151282.0
4       131881.0
          ...   
1479     46435.0
1480     33636.0
1481     29986.0
1482     74748.0
1483    240219.0
Name: School-Aged Population, Length: 1484, dtype: float64

In [5]:
# Simple preprocessing. Not going to worry about scaling right now. Just attempting to determine if my data is
# strong enough to warrant continuing.

cat_cols = []

for i in merged_df_pre_omi.columns:
    if i == 'Gene Copies (N1/L)' or i == 'Test Date':
        pass
    elif merged_df_pre_omi[i].dtype == 'object':
        cat_cols.append(i)
    elif merged_df_pre_omi[i].dtype == 'float64' or 'int64':
        print(i)
    else:
        print("error")


cat_transformer = Pipeline(steps=[  
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))                     
])
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols),
    ])

y = merged_df_pre_omi['Gene Copies (N1/L)']

X = merged_df_pre_omi.drop('Gene Copies (N1/L)', axis=1)

X_train ,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


Per Capita Gene Copies
Population Served, estimated
School-Aged Population
Holiday


In [6]:
# Setting up columns to be onehot encoded. We will look at numerical transformation later




# Have to drop the NaNs, otherwise linear model won't work
y_train = y_train.dropna()
X_train = X_train.dropna()
X_test = X_test.dropna()
y_test = y_test.dropna()

In [7]:
linreg = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

linreg.fit(X_train, y_train)


y_pred=linreg.predict(X_test)

linreg.score(X_train,y_train)



0.5587991188685084

In [8]:
linear_model = linreg.named_steps['model']

# Print the coefficients along with column names
for feature_name, coef in zip(X.columns, linear_model.coef_):
    print(f"{feature_name}: {coef}")

Test Date: -1070.2982270060952
WRRF Name: 1000.1580037146778
Per Capita Gene Copies: 1133.043333951316
Population Served, estimated: -729.9289403163428
School Status: 3534.929886399773
School-Aged Population: -1595.292150257837
Season: -2604.654484580207
Holiday: 1246.104726306294


In [9]:
# Let's try again but with log-transformed targets
y_train_2 = np.log(y_train)
y_test_2 = np.log(y_test)

In [10]:
linreg_2 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

linreg_2.fit(X_train, y_train_2)

y_pred_2=linreg_2.predict(X_test)

linreg_2.score(X_train,y_train_2)



0.631971730326365

In [11]:
linear_model = linreg_2.named_steps['model']

# Print the coefficients along with column names
for feature_name, coef in zip(X.columns, linear_model.coef_):
    print(f"{feature_name}: {coef}")

Test Date: -0.0440007780691526
WRRF Name: 0.20732123218866916
Per Capita Gene Copies: 0.19162232107698013
Population Served, estimated: -0.14624733932704054
School Status: 0.31736815887438546
School-Aged Population: -0.1297000508508879
Season: -0.3831365358061128
Holiday: 0.4055824921338149


In [12]:
# With the addition of jewish holidays, got a 1% boost in the r2 score. Might get a little nudge with islamic, but
# probably not much.

In [13]:
# Trying without the highly-correlated features

copy_df = merged_df_pre_omi.copy()

copy_df.dropna(inplace=True)

y = copy_df['Gene Copies (N1/L)']


X = copy_df.drop(columns=['Gene Copies (N1/L)','Population Served, estimated', 'Per Capita Gene Copies'], axis=1)

# Split again
X_train ,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


# Log transforming the target

y_test = np.log(y_test)
y_train = np.log(y_train)

In [14]:
# 3rd iteration, no adjustments to the model itself

linreg_3 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

linreg_3.fit(X_train, y_train)

y_pred = linreg_3.predict(X_test)

linreg_3.score(X_train,y_train)

0.6177258476117596

In [15]:
linear_model = linreg_3.named_steps['model']

# Print the coefficients along with column names
for feature_name, coef in zip(X.columns, linear_model.coef_):
    print(f"{feature_name}: {coef}")

Test Date: -0.14757150645203634
WRRF Name: 0.17730798873563733
School Status: 0.21434747417862648
School-Aged Population: -0.1206607271795389
Season: 0.28797817715926266
Holiday: -0.1298456319366709


In [16]:
# Slightly worse score without the non-school population data. So even though it's correlated, the general population
# data is not boosting the score much. 

In [17]:
# Let's look at just the school-related data.

copy_df_2 = merged_df_pre_omi.copy()
copy_df_2.dropna(inplace=True)

y = copy_df_2['Gene Copies (N1/L)']

X = copy_df_2[['School-Aged Population', 'School Status']]


# We only have one column to transform in this version

cat_cols_2 = ['School Status']

preprocessor_2 = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols_2),
    ])

X_train ,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [18]:
# 4th model - this one is actually different!

linreg_4 = Pipeline([
    ('preprocessor', preprocessor_2),
    ('model', LinearRegression())
])

linreg_4.fit(X_train, y_train)

y_pred = linreg_4.predict(X_test)

linreg_4.score(X_train,y_train)

0.46997170371988195

In [19]:
# Ok, so with just the school-aged population and school status, our model explains nearly half the variance! And this
# is before log transformation!


In [20]:
# Same model, log-transformed target data.

y_train = np.log(y_train)
y_test = np.log(y_test)

linreg_5 = Pipeline([
    ('preprocessor', preprocessor_2),
    ('model', LinearRegression())
])

linreg_5.fit(X_train, y_train)

y_pred = linreg_5.predict(X_test)

print(linreg_5.score(X_train,y_train))
print(r2_score(y_test, y_pred))

0.5422711234817856
0.5143954687950946


In [21]:
# Ok, our train tests scores are pretty close! Not too much overfitting.


In [22]:
# New X and y

copy_df_3 = merged_df_pre_omi.copy()
copy_df_3.dropna(inplace=True)
y = copy_df_3['Gene Copies (N1/L)']

X = copy_df_3['School-Aged Population']


# Split again

X_train ,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# log transform again

y_train = np.log(y_train)
y_test = np.log(y_test)

In [23]:
# using ols because we are not preprocessing this data


X_int = sm.add_constant(X_train)
results = sm.OLS(y_train, X_int).fit()
summary = results.summary()
print(summary)

influence = OLSInfluence(results)
print(influence.resid_studentized)

# well, this isn't very good! This is just school-aged population, though, and not the actual status of schools

                            OLS Regression Results                            
Dep. Variable:     Gene Copies (N1/L)   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     8.238
Date:                Sun, 30 Jul 2023   Prob (F-statistic):            0.00418
Time:                        17:18:11   Log-Likelihood:                -1784.3
No. Observations:                1097   AIC:                             3573.
Df Residuals:                    1095   BIC:                             3583.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      8

In [24]:
# Wonder how predictive the baseline model is. Let's just look at the original population data, which is one of
# the most relevant features. 

copy_df_4 = merged_df_pre_omi.copy()
copy_df_4.dropna(inplace=True)


y = copy_df_4['Gene Copies (N1/L)']

X = copy_df_4['Population Served, estimated']

X_train ,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train = np.log(y_train)
y_test = np.log(y_test)

In [25]:
X_int = sm.add_constant(X_train)
model_2 = sm.OLS(y_train, X_int).fit()
summary = model_2.summary()
summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Gene Copies (N1/L)   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     11.33
Date:                Sun, 30 Jul 2023   Prob (F-statistic):           0.000791
Time:                        17:18:12   Log-Likelihood:                -1782.7
No. Observations:                1097   AIC:                             3569.
Df Residuals:                    1095   BIC:                             3579.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            8.0670      0.075    106.995      0.000       7.919       8.215
Population Served, estimated  3.618e-07   1.08e-07      3.365      0.001    1.51e-07    5.73e-07
==============================================================================
Omnibus:                       61.143   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               38.686
Skew:                          -0.330   Prob(JB):                     3.98e-09
Kurtosis:                       2.359   Cond. No.                     1.42e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
copy_df_5 = merged_df_pre_omi.copy()
copy_df_5.reset_index()
copy_df_5.dropna(inplace=True)
y = copy_df_5['Gene Copies (N1/L)']

X = copy_df_5[['Population Served, estimated', 'WRRF Name', 'Per Capita Gene Copies', 'Test Date']]

X_train ,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


y_train = np.log(y_train)
y_test = np.log(y_test)

In [27]:
cat_cols_3 = ['WRRF Name']

cat_transformer = Pipeline(steps=[  
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))                     
])

preprocessor_3 = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols_3),
    ])

linreg_6 = Pipeline([
    ('preprocessor', preprocessor_3),
    ('model', LinearRegression())
])

linreg_6.fit(X_train, y_train)

y_pred = linreg_6.predict(X_test)

print(linreg_6.score(X_train,y_train))
print(r2_score(y_test, y_pred))

0.07123753415992828
0.04505102157812135


In [28]:
# So yeah, the basic info from the original dataset explains almost nothing. 

In [29]:
# Want to use the index (sample date) as a feature. Let's also save this data file as a csv, since
# it's what we ultimately want to use.

sample_date_df = merged_df_pre_omi.copy()

# sample_date_df.to_csv('data/master_wastewater.csv', index=False)

sample_date_df = sample_date_df.reset_index()
sample_date_df

#(Ending up abandoning dates at features, but in future work would like to integrate them.)

,index,Test Date,WRRF Name,Gene Copies (N1/L),Per Capita Gene Copies,"Population Served, estimated",School Status,School-Aged Population,Season,Holiday
0,0,2020-09-01,26th Ward,389.0,263535.64,290608,Summer Break_1,72091.0,Summer 2020,0
1,1,2020-09-01,Bowery Bay,1204.0,443632.86,924695,Summer Break_1,122994.0,Summer 2020,0
2,2,2020-09-01,Coney Island,304.0,168551.56,682342,Summer Break_1,117865.0,Summer 2020,0
3,3,2020-09-01,Hunts Point,940.0,574446.57,755948,Summer Break_1,151282.0,Summer 2020,0
4,4,2020-09-01,Jamaica Bay,632.0,233077.74,748737,Summer Break_1,131881.0,Summer 2020,0
...,...,...,...,...,...,...,...,...,...,...
1479,1479,2021-10-27,Port Richmond,881.0,560091.38,226167,Mandatory_In-person,46435.0,Fall 2021,0
1480,1480,2021-10-27,Red Hook,250.0,392726.56,224029,Mandatory_In-person,33636.0,Fall 2021,0
1481,1481,2021-10-27,Rockaway,1052.0,991360.72,120539,Mandatory_In-person,29986.0,Fall 2021,0
1482,1482,2021-10-27,Tallman Island,336.0,387871.66,449907,Mandatory_In-person,74748.0,Fall 2021,0


In [30]:
# Fancier models! Going to move on to more elaborate models, including Random Forest.

# Re-doing the columns for preprocessing since we have different features

date_cols = []
cat_cols = []  
num_cols = []   

for i in sample_date_df.columns:
    if i == 'Gene Copies (N1/L)' or i == 'Holiday':
        pass
    elif sample_date_df[i].dtype == 'datetime64[ns]':
        date_cols.append(i)
    elif sample_date_df[i].dtype == 'object':
        cat_cols.append(i)
    elif i == 'Sample Date':
        date_cols.append(i)
    elif sample_date_df[i].dtype == 'float64' or sample_date_df[i].dtype == 'int64':
        num_cols.append(i)
    else:
        print("error")
        


print(date_cols)
print(cat_cols)
print(num_cols)
# leaving "holiday" out because we don't want to transform this binary data

['Test Date']
['WRRF Name', 'School Status', 'Season']
['index', 'Per Capita Gene Copies', 'Population Served, estimated', 'School-Aged Population']


In [31]:
# Allows for fit and transformation of linear features in a pipeline. Were originally trying to combine
# LinearRegressor with RandomForest using FeatureUnion, but was unsuccessful

class ColumnSelector(BaseEstimator, TransformerMixin): 
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]
    

In [32]:
# New X and y, split

sample_copy_df = sample_date_df.copy()
sample_copy_df.dropna(inplace=True)

X = sample_copy_df.drop('Gene Copies (N1/L)', axis=1)

y = sample_copy_df['Gene Copies (N1/L)']


X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train = np.log(y_train)
y_test = np.log(y_test)


In [33]:
# New preprocessors

cat_preprocessor = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_preprocessor = Pipeline([
    ('selector', ColumnSelector(columns=num_cols)), 
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_cols),
    ('num', numeric_preprocessor, num_cols),
])


rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model',  RandomForestRegressor(random_state = 42))
])





In [34]:
# Just going with a vanilla Random Forest model after trying to combine a linear regression and random forest.


rf_pipeline.fit(X_train, y_train)


y_pred = rf_pipeline.predict(X_test)

print(f'R-squared score on training data: {rf_pipeline.score(X_train, y_train)}')
print(f'R2 test score: {r2_score(y_test, y_pred)}')
print(f'Mean squared error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean absolute error: {mean_absolute_error(y_test, y_pred)}')
print(f'Mean absolute percentage error: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'Explained variance score (modified R2): {explained_variance_score(y_test, y_pred)}')

R-squared score on training data: 0.9956072253052803
R2 test score: 0.9710455736297898
Mean squared error: 0.046502273870293584
Mean absolute error: 0.1557950212544598
Mean absolute percentage error: 0.019436302423708574
Explained variance score (modified R2): 0.9711137033002152


In [35]:
# Wow, 99.5%! 

In [36]:
# Let's look at feature importances:

rf_model = rf_pipeline.named_steps['model']
features = []
scores = []

# Print the coefficients along with column names
for feature_name, importance in zip(X.columns, rf_model.feature_importances_):
    features.append(feature_name)
    scores.append(importance)
    
ranked_scores = sorted(zip(scores, features), reverse=True)
for score, feature in ranked_scores:
    print(f'{score}: {feature}')

0.0032408855024702197: index
0.0012653541334829968: Holiday
0.0012030764686354338: Test Date
0.0009657866936284619: School-Aged Population
0.0009570865532883691: Season
0.0007299278255275428: Population Served, estimated
0.0005533291436602843: Per Capita Gene Copies
0.0003931522738028303: WRRF Name
0.00018733503013810266: School Status


In [37]:
# Cross-validating, because we ultimately want to use this on other data!
cross_validate(rf_pipeline, X_train, y_train, return_train_score=True)

{'fit_time': array([0.9377737 , 0.91335988, 1.04615211, 0.95964003, 0.92230296]),
 'score_time': array([0.01658916, 0.01693606, 0.01571393, 0.01702785, 0.01662803]),
 'test_score': array([0.97500962, 0.96817421, 0.96257558, 0.9672737 , 0.95461637]),
 'train_score': array([0.99503409, 0.99527155, 0.99554904, 0.99514594, 0.99563054])}

In [38]:
# Want to try this model again without the date features, since they seem to be over-determining.

sample_copy_df_2 = sample_date_df.copy()

sample_copy_df_2.dropna(inplace=True)

X = sample_copy_df_2.drop(['Gene Copies (N1/L)', 'Sample Date', 'Test Date'], axis=1)

y = sample_copy_df_2['Gene Copies (N1/L)']

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train = np.log(y_train)
y_test = np.log(y_test)

KeyError: "['Sample Date'] not found in axis"

In [ ]:
rf_pipeline.fit(X_train, y_train)  # Same pipeline, different data


y_pred = rf_pipeline.predict(X_test)


print(f'R-squared score on training data: {rf_pipeline.score(X_train, y_train)}')
print(f'R2 test score: {r2_score(y_test, y_pred)}')
print(f'Mean squared error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean absolute error: {mean_absolute_error(y_test, y_pred)}')
print(f'Mean absolute percentage error: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'Explained variance score (modified R2): {explained_variance_score(y_test, y_pred)}')

In [ ]:
# The very same! Our engineered features are very strong all around. Let's look at feature importance.

rf_model = rf_pipeline.named_steps['model']
features = []
scores = []
# Print the coefficients along with column names
for feature_name, importance in zip(X.columns, rf_model.feature_importances_):
    features.append(feature_name)
    scores.append(importance)
    
ranked_scores = sorted(zip(scores, features), reverse=True)
for score, feature in ranked_scores:
    print(f'{score}: {feature}')

In [ ]:
# Let's try another model type, using all features

sample_copy_df = sample_date_df.copy()

X = sample_copy_df.drop('Gene Copies (N1/L)', axis=1)

y = sample_copy_df['Gene Copies (N1/L)']

X.dropna(inplace=True)
y.dropna(inplace=True)

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train = np.log(y_train)
y_test = np.log(y_test)

In [ ]:

grad = GradientBoostingRegressor()
grad_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model',  grad)
])


grad_pipeline.fit(X_train, y_train)
y_pred = grad_pipeline.predict(X_test)

print(f'R-squared score on training data: {grad_pipeline.score(X_train, y_train)}')
print(f'R2 test score: {r2_score(y_test, y_pred)}')
print(f'Mean squared error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean absolute error: {mean_absolute_error(y_test, y_pred)}')
print(f'Mean absolute percentage error: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'Explained variance score (modified R2 test score): {explained_variance_score(y_test, y_pred)}')

In [ ]:
# Pretty similar!

In [ ]:
grad_model = grad_pipeline.named_steps['model']
features = []
scores = []
# Print the coefficients along with column names
for feature_name, importance in zip(X.columns, grad_model.feature_importances_):
    features.append(feature_name)
    scores.append(importance)
    
ranked_scores = sorted(zip(scores, features), reverse=True)
for score, feature in ranked_scores:
    print(f'{score}: {feature}')

In [ ]:
# Very different weighing of features here, and slightly better test score. Let's look at loss scores, too:
train_score = grad_pipeline['model'].train_score_
train_score


# If we ran maybe double the iteratations, our loss score should approach .2

In [ ]:
# Let's get rid of all date features with this model and see what happens

sample_copy_df = sample_date_df.copy()

X = sample_copy_df.drop(columns=['Gene Copies (N1/L)','Sample Date', 'Test Date'], axis=1)

y = sample_copy_df['Gene Copies (N1/L)']

X.dropna(inplace=True)
y.dropna(inplace=True)

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train = np.log(y_train)
y_test = np.log(y_test)

grad = GradientBoostingRegressor()
grad_best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model',  grad)
])

grad_best_pipeline.fit(X_train, y_train)
y_pred = grad_best_pipeline.predict(X_test)

print(f'R-squared score on training data: {grad_best_pipeline.score(X_train, y_train)}')
print(f'R2 test score: {r2_score(y_test, y_pred)}')
print(f'Mean squared error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean absolute error: {mean_absolute_error(y_test, y_pred)}')
print(f'Mean absolute percentage error: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'Explained variance score (modified R2 test score): {explained_variance_score(y_test, y_pred)}')

In [ ]:
# Very similar scores, with less over-fitting. Good! # Let's examine feature importance.

grad_model = grad_best_pipeline.named_steps['model']
features = []
scores = []
# Print the coefficients along with column names
for feature_name, importance in zip(X.columns, grad_model.feature_importances_):
    features.append(feature_name)
    scores.append(importance)
    
ranked_scores = sorted(zip(scores, features), reverse=True)
for score, feature in ranked_scores:
    print(f'{score}: {feature}')

In [ ]:
# Going to remove highly-correlated non-engineered features, to better see how strong the model is with my contibutions.

sample_copy_df = sample_date_df.copy()

sample_copy_df.dropna(inplace=True)

X = sample_copy_df.drop(columns=['Gene Copies (N1/L)','Sample Date', 'Test Date', 'Population Served, estimated', 'Per Capita Gene Copies'], axis=1)

y = sample_copy_df['Gene Copies (N1/L)']

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train = np.log(y_train)
y_test = np.log(y_test)

In [ ]:
# Re-do preprocesing as we have different features
date_cols = []
cat_cols = []  
num_cols = []   

for i in sample_date_df.columns:
    if i == 'Gene Copies (N1/L)' or i == 'Holiday' or i == 'Population Served, estimated' or i == 'Per Capita Gene Copies':
        pass
    elif sample_date_df[i].dtype == 'datetime64[ns]':
        date_cols.append(i)
    elif sample_date_df[i].dtype == 'object':
        cat_cols.append(i)
    elif i == 'Sample Date':
        date_cols.append(i)
    elif sample_date_df[i].dtype == 'float64' or sample_date_df[i].dtype == 'int64':
        num_cols.append(i)
    else:
        print("error")
        


print(date_cols)
print(cat_cols)
print(num_cols)
# leaving "holiday" out because we don't want to transform this binary data

In [ ]:
cat_transformer = Pipeline(steps=[  
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))                     
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols),
    ])

grad = GradientBoostingRegressor()
grad_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model',  grad)
])

grad_pipeline.fit(X_train, y_train)
y_pred = grad_pipeline.predict(X_test)

print(f'R-squared score on training data: {grad_pipeline.score(X_train, y_train)}')
print(f'R2 test score: {r2_score(y_test, y_pred)}')
print(f'Mean squared error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean absolute error: {mean_absolute_error(y_test, y_pred)}')
print(f'Mean absolute percentage error: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'Explained variance score (modified R2 test score): {explained_variance_score(y_test, y_pred)}')

In [ ]:
# Looking at feature importances

grad_model = grad_pipeline.named_steps['model']
features = []
scores = []
# Print the coefficients along with column names
for feature_name, importance in zip(X.columns, grad_model.feature_importances_):
    features.append(feature_name)
    scores.append(importance)
    
ranked_scores = sorted(zip(scores, features), reverse=True)
for score, feature in ranked_scores:
    print(f'{score}: {feature}')

In [ ]:
# By removing correlated features, our score was reduced pretty dramatically. However, there seem to be important
# elements of the non-date correlated features that we should keep, because although they are similar, they tell us
# important things about the data at particular times. For instance, sudden wastewater spikes around school events
# in places with high school-aged populations. Over time, these effects are expected to flatten out and/or become
# more cyclical. One avenue to pursue later would be population estimates of those who left the city during 2020/2021
# but who were never officially non-residents, and thus not reflected in the Census Bureau's data. We would expect much more of this in wealthier zip codes and in zip codes
# where there are fewer children. 

In [ ]:
# Setting up for grid search of our best model so far. Re-doing preprocessing since using different features

date_cols = []
cat_cols = []  
num_cols = []   

for i in sample_date_df.columns:
    if i == 'Gene Copies (N1/L)' or i == 'Holiday':
        pass
    elif sample_date_df[i].dtype == 'datetime64[ns]':
        date_cols.append(i)
    elif sample_date_df[i].dtype == 'object':
        cat_cols.append(i)
    elif i == 'Sample Date':
        date_cols.append(i)
    elif sample_date_df[i].dtype == 'float64' or sample_date_df[i].dtype == 'int64':
        num_cols.append(i)
    else:
        print("error")
        
print(date_cols)
print(cat_cols)
print(num_cols)

cat_preprocessor = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_preprocessor = Pipeline([
    ('selector', ColumnSelector(columns=num_cols)), 
    ('scaler', StandardScaler())
])


preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_cols),
    ('num', numeric_preprocessor, num_cols),
])


In [ ]:
# Removing the datetime info and re-running grid search on our best model.

sample_copy_df = sample_date_df.copy()
sample_copy_df.dropna(inplace=True)

X = sample_copy_df.drop(columns=['Gene Copies (N1/L)','Sample Date', 'Test Date'], axis=1)

y = sample_copy_df['Gene Copies (N1/L)']


X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train = np.log(y_train)
y_test = np.log(y_test)
grad = GradientBoostingRegressor()

param_grid = {
    'model__learning_rate': [.01, .1, .3],
    'model__loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'model__n_estimators': [100, 150, 200, 250],
    'model__subsample': [.3, .5, .7, 1.0],
    'model__criterion': ['friedman_mse', 'squared_error']
    
}
                            
grid_search_gbc = GridSearchCV(
    estimator = grad_best_pipeline,  # pipeline 
    param_grid = param_grid,
    cv= 5,
    scoring='explained_variance'  # internal scoring term
)

grid_search_gbc.fit(X_train, y_train)

cv_score = grid_search_gbc.best_score_
test_score = r2_score(y_test, grid_search_gbc.predict(X_test))

print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
print(grid_search_gbc.best_params_)

In [ ]:
# Thought I was done, but want to try one more model type: LinearTrees!

sample_copy_df = sample_date_df.copy()

sample_copy_df.dropna(inplace=True)

X = sample_copy_df.drop(columns=['Gene Copies (N1/L)','Sample Date', 'Test Date'], axis=1)

y = sample_copy_df['Gene Copies (N1/L)']

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


y_train = np.log(y_train)
y_test = np.log(y_test)

In [116]:

lintree = LinearTreeRegressor(base_estimator=LinearRegression())

lintree_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('to_sparse', FunctionTransformer(csr_matrix, validate=False)),
    ('to_dense', FunctionTransformer(lambda x: x.toarray(), validate=False)),
    ('model',  lintree)
])

lintree_pipeline.fit(X_train, y_train)

y_pred = lintree_pipeline.predict(X_test)

print(f'R-squared score on training data: {lintree_pipeline.score(X_train, y_train)}')
print(f'R2 test score: {r2_score(y_test, y_pred)}')
print(f'Mean squared error: {mean_squared_error(y_test, y_pred)}')
print(f'Mean absolute error: {mean_absolute_error(y_test, y_pred)}')
print(f'Mean absolute percentage error: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'Explained variance score (modified R2 test score): {explained_variance_score(y_test, y_pred)}')

# Well, that was a terrible model for this data, but seemed interesting! 

R-squared score on training data: 0.9802052054633759
R2 test score: -1184.0720230715035
Mean squared error: 1903.285634751599
Mean absolute error: 3.5056331972161354
Mean absolute percentage error: 0.4220202108653636
Explained variance score (modified R2 test score): -1177.0278641925847
